# Traductor de texto japonés a español

In [ ]:
# Install the transformers library from HuggingFace
#! pip install transformers torch pytesseract

In [ ]:
# Install the sentencepiece and sacremoses libraries to tokenize the text
#! pip install sentencepiece sacremoses

In [4]:
# ! pip install python-dotenv

In [1]:
# Install deep-translator to translate the text
! pip install deep-translator

# Google translate

In [ ]:
from xml.etree import ElementTree as ET
from deep_translator import GoogleTranslator
import os
import html  # Importa la biblioteca html

# Inicializa el traductor de Google del japonés al español
traductor = GoogleTranslator(source='ja', target='es')

# Cargar el archivo XML
xml_path = '/home/carol/code/cparran/MOJI/raw_data/ARMS/ARMS_lite.xml'  # Asegúrate de especificar la ruta correcta al archivo XML
tree = ET.parse(xml_path)
root = tree.getroot()

# Obtener el nombre del archivo original sin la extensión
file_name = os.path.splitext(os.path.basename(xml_path))[0]

# Iterar sobre cada elemento 'text'
for text_element in root.findall('.//text'):
    # Extraer el texto japonés
    japanese_text = text_element.text.strip()
    
    # Verificar que el texto no esté vacío antes de traducir
    if japanese_text:
        # Traduce el texto extraído al español
        translated_text = traductor.translate(japanese_text)
        
        # Verifica que translated_text no sea None antes de decodificar las entidades HTML
        if translated_text:
            # Decodificar las entidades HTML a caracteres normales
            translated_text = html.unescape(translated_text)
        else:
            # Manejar el caso en que translated_text es None
            # Por ejemplo, puedes optar por dejar el texto original o aplicar una etiqueta predeterminada
            translated_text = "Texto no disponible"
        
        # Opción 1: Agregar el texto traducido como un nuevo atributo
        text_element.set('translated_text', translated_text)
        
        # Opción 2 (Alternativa): Crear y agregar un nuevo elemento hijo con el texto traducido
        # Si prefieres esta opción, descomenta las siguientes dos líneas:
        # new_element = ET.SubElement(text_element, "translatedText")
        # new_element.text = translated_text

# Generar el nombre del archivo de salida con el sufijo "-es"
output_file_name = f"{file_name}-es.xml"

# Guardar el archivo XML modificado, asegurándose de usar utf-8 como la codificación
tree.write(output_file_name, encoding="utf-8", xml_declaration=True)

In [ ]:
# LLama a la función traducir_archivo_xml

from traductor_xml import traducir_archivo_xml()

xml_input_path = '/home/carol/code/cparran/MOJI/raw_data/ARMS/ARMS_lite.xml'
traducir_archivo_xml(xml_input_path)

# Deepl Translator

Es más confiable

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from xml.etree import ElementTree as ET
import html
import os
from deep_translator import DeeplTranslator

# Asume que esta función se define en tu entorno con la clave API configurada
traductor = DeeplTranslator(api_key="52869b3f-cf74-420c-8994-b697d207234e:fx", source="ja", target="es", use_free_api=True)

def traducir_texto(texto):
    traduccion = traductor.translate(texto)
    return html.unescape(traduccion)

def traducir_archivo_xml_concurrente(xml_path, carpeta_destino):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    textos_para_traducir = [elem.text.strip() for elem in root.findall('.//text') if elem.text and elem.text.strip()]

    # Usar ThreadPoolExecutor para traducir en paralelo
    with ThreadPoolExecutor(max_workers=10) as executor:
        resultados = list(executor.map(traducir_texto, textos_para_traducir))
    
    for elem, traduccion in zip(root.findall('.//text'), resultados):
        if traduccion:
            elem.set('translated_text', traduccion)
    
    # Construye la ruta del archivo de salida usando la carpeta destino especificada
    file_name = os.path.splitext(os.path.basename(xml_path))[0]
    output_file_name = os.path.join(carpeta_destino, f"{file_name}-es.xml")
    
    # Guarda el archivo XML traducido
    tree.write(output_file_name, encoding="utf-8", xml_declaration=True)
    print(f"Archivo traducido guardado en: {output_file_name}")

# Ejemplo de cómo llamar a la función
xml_input_path = '/home/carol/code/cparran/MOJI/raw_data/ARMS/ARMS_lite.xml'
carpeta_destino = '/home/carol/code/cparran/MOJI/raw_data/output'
traducir_archivo_xml_concurrente(xml_input_path, carpeta_destino)


# DeepL Translator (más rápido)

Sin eliminar definiciones

In [ ]:
# from xml.etree import ElementTree as ET
# from deep_translator import DeeplTranslator
# import os
# import html
# from concurrent.futures import ThreadPoolExecutor

# # Asume que esta función se define en tu entorno con la clave API configurada
# api_key = "52869b3f-cf74-420c-8994-b697d207234e:fx"  # Reemplaza con tu clave de API real
# traductor = DeeplTranslator(api_key=api_key, source="ja", target="es", use_free_api=True)

# def traducir_texto(texto):
#     if texto:
#         traduccion = traductor.translate(texto)
#         return html.unescape(traduccion)
#     return ""

# def traducir_archivo_xml_concurrente(xml_path, output_folder):
#     tree = ET.parse(xml_path)
#     root = tree.getroot()

#     # Preparar los textos para traducción
#     textos_para_traducir = [(elem, elem.text.strip()) for elem in root.findall('.//text') if elem.text and elem.text.strip()]
    
#     # Utilizar ThreadPoolExecutor para traducir en paralelo
#     with ThreadPoolExecutor(max_workers=20) as executor:
#         # Mapear textos a la función de traducción y actualizar elementos XML con traducciones
#         for elem, traduccion in zip(textos_para_traducir, executor.map(lambda x: traducir_texto(x[1]), textos_para_traducir)):
#             elem[0].set('translated_text', traduccion)
    
#     # Construye el nombre del archivo de salida y guarda el archivo XML traducido
#     file_name = os.path.splitext(os.path.basename(xml_path))[0] + "-es.xml"
#     output_file_path = os.path.join(output_folder, file_name)
#     tree.write(output_file_path, encoding="utf-8", xml_declaration=True)
#     print(f"Archivo traducido guardado en: {output_file_path}")

# # Uso del código
# xml_input_path = '/home/carol/code/cparran/MOJI/raw_data/ARMS/ARMS_lite.xml'
# output_folder = '/home/carol/code/cparran/MOJI/raw_data/output'  # Asegúrate de especificar la carpeta de destino correcta
# traducir_archivo_xml_concurrente(xml_input_path, output_folder)


Elimina las explicaciones

In [31]:
from xml.etree import ElementTree as ET
from deep_translator import DeeplTranslator
import os
import html
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor
import re

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Obtener la clave API de DeepL desde las variables de entorno
api_key = os.getenv("DEEPL_API_KEY")

# Asume que esta función se define en tu entorno con la clave API configurada
api_key = api_key  # Asegúrate de manejar tus claves API de manera segura
traductor = DeeplTranslator(api_key=api_key, source="ja", target="es", use_free_api=True)

def eliminar_explicaciones(texto):
    # Eliminar texto dentro de paréntesis, incluyendo los paréntesis mismos
    texto_sin_explicaciones = re.sub(r'\([^)]*\)', '', texto)
    # Agregar más reglas de limpieza aquí si es necesario
    return texto_sin_explicaciones.strip()

def traducir_texto(texto):
    if texto:
        traduccion = traductor.translate(texto)
        traduccion_limpia = html.unescape(traduccion)
        traduccion_final = eliminar_explicaciones(traduccion_limpia)
        return traduccion_final
    return ""

def traducir_archivo_xml_concurrente(xml_path, output_folder):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    textos_para_traducir = [(elem, elem.text.strip()) for elem in root.findall('.//text') if elem.text and elem.text.strip()]
    
    with ThreadPoolExecutor(max_workers=20) as executor:
        for elem, traduccion in zip(textos_para_traducir, executor.map(lambda x: traducir_texto(x[1]), textos_para_traducir)):
            elem[0].set('translated_text', traduccion)
    
    file_name = os.path.splitext(os.path.basename(xml_path))[0] + "-es.xml"
    output_file_path = os.path.join(output_folder, file_name)
    tree.write(output_file_path, encoding="utf-8", xml_declaration=True)
    print(f"Archivo traducido guardado en: {output_file_path}")

xml_input_path = '/home/carol/code/cparran/MOJI/raw_data/ARMS/ARMS_lite.xml'
output_folder = '/home/carol/code/cparran/MOJI/raw_data/output'
traducir_archivo_xml_concurrente(xml_input_path, output_folder)


Archivo traducido guardado en: /home/carol/code/cparran/MOJI/raw_data/output/ARMS_lite-es.xml


# OPEN AI Translator

In [59]:
#! pip install openai==0.27.0
! pip install deep-translator==1.9.0

  Attempting uninstall: deep-translator
    Found existing installation: deep-translator 1.11.4
    Uninstalling deep-translator-1.11.4:
      Successfully uninstalled deep-translator-1.11.4


In [64]:
from xml.etree import ElementTree as ET
import openai
import os
import html
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor
import re

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Obtener la clave API de OpenAI desde las variables de entorno
openai_api_key = os.getenv("OPENAI_API_KEY")

# Configurar la API de OpenAI
openai.api_key = openai_api_key

def eliminar_explicaciones(texto):
    # Eliminar texto dentro de paréntesis, incluyendo los paréntesis mismos
    texto_sin_explicaciones = re.sub(r'\([^)]*\)', '', texto)
    return texto_sin_explicaciones.strip()

def traducir_texto(texto):
    if not texto:
        return ""

    # Traducción con GPT-3
    traduccion = openai.Completion.create(
        engine="text-davinci-002",
        prompt=texto,
        max_tokens=100
    )['choices'][0]['text']
    
    traduccion_limpia = html.unescape(traduccion)
    
    return eliminar_explicaciones(traduccion_limpia)

def traducir_archivo_xml_concurrente(xml_path, output_folder):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    textos_para_traducir = [(elem, elem.text.strip()) for elem in root.findall('.//text') if elem.text and elem.text.strip()]
    
    with ThreadPoolExecutor(max_workers=20) as executor:
        for elem, traduccion in zip(textos_para_traducir, executor.map(lambda x: traducir_texto(x[1]), textos_para_traducir)):
            elem[0].set('translated_text', traduccion)
    
    file_name = os.path.splitext(os.path.basename(xml_path))[0] + "-es.xml"
    output_file_path = os.path.join(output_folder, file_name)
    tree.write(output_file_path, encoding="utf-8", xml_declaration=True)
    print(f"Archivo traducido guardado en: {output_file_path}")

xml_input_path = '/home/carol/code/cparran/MOJI/raw_data/ARMS/ARMS_lite.xml'
output_folder = '/home/carol/code/cparran/MOJI/raw_data/output'
traducir_archivo_xml_concurrente(xml_input_path, output_folder)


APIRemovedInV1: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


# Combina Google Translate y DeepL

In [41]:
from xml.etree import ElementTree as ET
from deep_translator import GoogleTranslator, DeeplTranslator
import os
import html
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor
import re

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Obtener la clave API de DeepL desde las variables de entorno
api_key = os.getenv("DEEPL_API_KEY")

def eliminar_explicaciones(texto):
    # Eliminar texto dentro de paréntesis, incluyendo los paréntesis mismos
    texto_sin_explicaciones = re.sub(r'\([^)]*\)', '', texto)
    return texto_sin_explicaciones.strip()

def traducir_texto(texto):
    if not texto:
        return ""
    
    # Traducción con Google Translator
    traduccion_google = GoogleTranslator(source='ja', target='es').translate(texto)
    traduccion_google_limpia = html.unescape(traduccion_google)

    # Traducción con DeepL
    traduccion_deepl_respuesta = DeeplTranslator(api_key=api_key, source="ja", target="es", use_free_api=True).translate(texto, return_all=True)
    
    # Manejar el caso en que la respuesta no es como se esperaba
    if 'translations' in traduccion_deepl_respuesta and traduccion_deepl_respuesta['translations']:
        traduccion_deepl_limpia = html.unescape(traduccion_deepl_respuesta['translations'][0]['text'])
        
        # Comparar las traducciones por la puntuación de confianza de DeepL y seleccionar la más alta
        if 'confidence' in traduccion_deepl_respuesta['translations'][0] and traduccion_deepl_respuesta['translations'][0]['confidence'] > 0.5:
            mejor_traduccion = traduccion_deepl_limpia
        else:
            mejor_traduccion = traduccion_google_limpia
    else:
        # En caso de una respuesta inesperada, utilizar la traducción de Google como fallback
        mejor_traduccion = traduccion_google_limpia
    
    return eliminar_explicaciones(mejor_traduccion)

def traducir_archivo_xml_concurrente(xml_path, output_folder):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    textos_para_traducir = [(elem, elem.text.strip()) for elem in root.findall('.//text') if elem.text and elem.text.strip()]
    
    with ThreadPoolExecutor(max_workers=20) as executor:
        futures = [executor.submit(traducir_texto, texto[1]) for texto in textos_para_traducir]
        for future, elem in zip(futures, textos_para_traducir):
            elem[0].set('translated_text', future.result())
    
    file_name = os.path.splitext(os.path.basename(xml_path))[0] + "-es.xml"
    output_file_path = os.path.join(output_folder, file_name)
    tree.write(output_file_path, encoding="utf-8", xml_declaration=True)
    print(f"Archivo traducido guardado en: {output_file_path}")

# Definir la ruta de entrada del archivo XML y la carpeta de salida
xml_input_path = '/home/carol/code/cparran/MOJI/raw_data/ARMS/ARMS_lite.xml'
output_folder = '/home/carol/code/cparran/MOJI/raw_data/output'

# Ejecutar la función principal
traducir_archivo_xml_concurrente(xml_input_path, output_folder)


Archivo traducido guardado en: /home/carol/code/cparran/MOJI/raw_data/output/ARMS_lite-es.xml


# Si la traducción de Google no es confiable, usa DeepL

In [51]:
#! pip install langid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 4.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941172 sha256=06c0a80475046a718e24694b4ce85f018169337444ccb0eeff05bc007264a282
  Stored in directory: /home/carol/.cache/pip/wheels/23/c8/c6/eed80894918490a175677414d40bd7c851413bbe03d4856c3c
Successfully built langid


In [66]:
from xml.etree import ElementTree as ET
from deep_translator import GoogleTranslator, DeeplTranslator
import os
import html
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor
import re
import langid

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Obtener la clave API de DeepL desde las variables de entorno
api_key = os.getenv("DEEPL_API_KEY")

def eliminar_explicaciones(texto):
    # Eliminar texto dentro de paréntesis, incluyendo los paréntesis mismos
    texto_sin_explicaciones = re.sub(r'\([^)]*\)', '', texto)
    return texto_sin_explicaciones.strip()

def detectar_idioma(texto):
    # Utilizar langid para detectar el idioma del texto
    idioma, _ = langid.classify(texto)
    return idioma

def traducir_texto(texto):
    if not texto:
        return ""

    # Detectar el idioma del texto
    idioma = detectar_idioma(texto)

    # Traducción con Google Translator si el idioma detectado es japonés (ja)
    if idioma == 'ja':
        traduccion_google = GoogleTranslator(source='ja', target='es').translate(texto)
        return eliminar_explicaciones(html.unescape(traduccion_google))

    # Traducción con DeepL para otros idiomas
    traduccion_deepl_respuesta = DeeplTranslator(api_key=api_key, source=idioma, target="es", use_free_api=True).translate(texto, return_all=True)
    
    if 'translations' in traduccion_deepl_respuesta and traduccion_deepl_respuesta['translations']:
        traduccion_deepl_texto = traduccion_deepl_respuesta['translations'][0]['text']
        return eliminar_explicaciones(html.unescape(traduccion_deepl_texto))
    else:
        # En caso de una respuesta inesperada, devolver el texto original
        return texto

def traducir_archivo_xml_concurrente(xml_path, output_folder):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    textos_para_traducir = [(elem, elem.text.strip()) for elem in root.findall('.//text') if elem.text and elem.text.strip()]
    
    with ThreadPoolExecutor(max_workers=20) as executor:
        futures = [executor.submit(traducir_texto, texto[1]) for texto in textos_para_traducir]
        for future, elem in zip(futures, textos_para_traducir):
            elem[0].set('translated_text', future.result())
    
    file_name = os.path.splitext(os.path.basename(xml_path))[0] + "-es.xml"
    output_file_path = os.path.join(output_folder, file_name)
    tree.write(output_file_path, encoding="utf-8", xml_declaration=True)
    print(f"Archivo traducido guardado en: {output_file_path}")

# Definir la ruta de entrada del archivo XML y la carpeta de salida
xml_input_path = '/home/carol/code/cparran/MOJI/raw_data/ARMS/ARMS_lite.xml'
output_folder = '/home/carol/code/cparran/MOJI/raw_data/output'

# Ejecutar la función principal
traducir_archivo_xml_concurrente(xml_input_path, output_folder)




Archivo traducido guardado en: /home/carol/code/cparran/MOJI/raw_data/output/ARMS_lite-es.xml


# Traduce y aplica criterio para seleccionar la mejor traducción

In [65]:
from xml.etree import ElementTree as ET
from deep_translator import GoogleTranslator, DeeplTranslator
import os
import html
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor
import re
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import pipeline

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Obtener la clave API de DeepL desde las variables de entorno
api_key = os.getenv("DEEPL_API_KEY")

# Inicializar el tokenizer y el modelo de BERT para la evaluación de coherencia
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

def evaluar_coherencia(texto):
    resultado = nlp(texto)
    return resultado[0]['score']  # Ejemplo simplificado

def seleccionar_mejor_traduccion(texto_original):
    traducciones = [
        html.unescape(GoogleTranslator(source='ja', target='es').translate(texto_original)),
        html.unescape(DeeplTranslator(api_key=api_key, source="ja", target="es", use_free_api=True).translate(texto_original))
    ]

    puntuaciones = [evaluar_coherencia(traduccion) for traduccion in traducciones]
    mejor_indice = puntuaciones.index(max(puntuaciones))

    return traducciones[mejor_indice]

def eliminar_explicaciones(texto):
    texto_sin_explicaciones = re.sub(r'\([^)]*\)', '', texto)
    return texto_sin_explicaciones.strip()

def traducir_texto(texto):
    if not texto:
        return ""
    
    mejor_traduccion = seleccionar_mejor_traduccion(texto)
    return eliminar_explicaciones(mejor_traduccion)

def traducir_archivo_xml_concurrente(xml_path, output_folder):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    textos_para_traducir = [(elem, elem.text.strip()) for elem in root.findall('.//text') if elem.text and elem.text.strip()]
    
    with ThreadPoolExecutor(max_workers=20) as executor:
        futures = [executor.submit(traducir_texto, texto[1]) for texto in textos_para_traducir]
        for future, elem in zip(futures, textos_para_traducir):
            elem[0].set('translated_text', future.result())
    
    file_name = os.path.splitext(os.path.basename(xml_path))[0] + "-es.xml"
    output_file_path = os.path.join(output_folder, file_name)
    tree.write(output_file_path, encoding="utf-8", xml_declaration=True)
    print(f"Archivo traducido guardado en: {output_file_path}")

xml_input_path = '/home/carol/code/cparran/MOJI/raw_data/ARMS/ARMS_lite.xml'
output_folder = '/home/carol/code/cparran/MOJI/raw_data/output'

traducir_archivo_xml_concurrente(xml_input_path, output_folder)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Archivo traducido guardado en: /home/carol/code/cparran/MOJI/raw_data/output/ARMS_lite-es.xml
